<a href="https://colab.research.google.com/github/ykitaguchi77/AutoGen/blob/main/Autogen%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB_%E3%82%B0%E3%83%AB%E3%83%BC%E3%83%97%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%EF%BC%88%E3%83%A1%E3%83%8B%E3%83%A5%E3%83%BC%E8%80%83%E6%A1%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogenについて
Microsoft Researchから登場したマルチエージェントによるLLMアプリケーションを実現するフレームワーク。役割を設定したエージェントを定義することで、エージェントがお互いに相談しながらタスクを実行できる。インターネットアクセスやコードの実行、ファンクションなども活用しながら高度な操作ができる。

- 公式サイト  
  https://microsoft.github.io/autogen/  
- Github  
  https://github.com/microsoft/autogen  
- ブログ  
  AutoGen: Enabling next-generation large language model applications  
  https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/  
- arxivペーパー  
  AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation  
  https://arxiv.org/abs/2308.08155  

## 注意
- 実行には、OpenAIのAPIキーが必要。
- 複雑な処理や高度な連携にはGPT-4のアクセスが必要。
- コードの修正などが繰り返されるケースではAPIアクセスが多発しコストが嵩むので注意。
- タスクによっては、エージェント間でコードやソースなどを共有する必要があり、大量の入力が発生するため、16Kや32Kに対応したモデルでないと動作しない（Rate Limitエラーにひっかかる）。
- Web情報の取得やデータ分析などは16K、32Kモデル必須。


### STEP 1：準備
pipでインストール。  
ランタイム環境にインストールされてない場合は以下実行。

In [1]:
%%capture --no-stderr
%pip install pyautogen~=0.1.0

### STEP2：設定
設定はOAI_CONFIGファイルから読み込むこともできるが、今回は直接設定する。  
"api_key"にOpenAIのAPIキーを登録しておく。  
Azureを利用する場合は本家のサンプルを参照のこと。  

In [2]:
# APIキー設定スクリプト
# 目的：Google Driveからapi.txtファイルを読み込み、必要なAPIキーを環境変数として設定する
# 使用するAPI：OpenAI API, SerpAPI, Google Custom Search Engine API

# APIの設定
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
#openai_api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]
os.environ["GOOGLE_CSE_ID"] = key[9]
os.environ["GOOGLE_API_KEY"] = key[11]

Mounted at /content/drive


In [3]:
import autogen

config_list = [
    {
        "model": "gpt-4o",
        "api_key": key[3],
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
#        "request_timeout": 120,
#        "max_retry_period": 90,
#        "retry_wait_time": 5,
    },
    {
        "model": "gpt-4o-mini",
        "api_key": key[3],
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
    },
]

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### エージェントの定義
今回の例では、以下の４つのエージェントを定義し、1週間分の夕食の献立を相談しながら決めてもらう。  
- floor_manager  
ユーザーとの対話を担当  
- chef  
メニューを考案する  
- doctor  
医学的立場からメニューを修正する  
- kitchen_manager  
メニューに必要な食材などを調達する
  
「system_message=」でエージェントの役割を定義する。  
  
今回はコードの生成と実行はしないが、必要な場合は  
「code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},」  
を利用する。  
  
「groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)」  
で、参加させるエージェントと実行ラウンド数を決める。  
  
人間の介入を受け付ける場合は、以下を変更
「human_input_mode="TERMINATE"」→「human_input_mode="ALWAYS"」

In [14]:
llm_config = {"config_list": config_list}
floor_manager = autogen.UserProxyAgent(
   name="floor_manager",
   system_message="ChefやDoctor、Kitchen_Managerと相談しながら調理や食材に関する課題を解決してください。満足が得られなければ再度フィードバックをかけてください",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
chef = autogen.AssistantAgent(
    name="chef",
    system_message="世界中の料理を知り尽くした料理人です。健康面はdoctorが検討するので、考慮する必要はありません。味についてのみ検討したレシピを考案し、最高の料理を提供します。考案したメニューをdoctorと相談して、健康面を考慮しながら修正してください。メニューが決定したらfloor_managerに食材リストの提出を依頼してください。",
    llm_config=llm_config,
)
doctor = autogen.AssistantAgent(
    name="doctor",
    system_message="chefが提案したメニューを医学的な立場で検証し、floor_managerに修正を依頼してください。",
    llm_config=llm_config,
)
kitchen_manager = autogen.AssistantAgent(
    name="kitchen_manager",
    system_message="料理にかかる費用や必要な食材や調味料を管理します。chefが考案しメニューから必要な食材を検討し、floor_managerに知らせてください。",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

### チャット開始
議論してもらうテーマを与えると議論が開始される。  
  
実行時に「chef(to chat_manager):」のように誰宛ての会話なのかが表示される点に注目。

In [15]:
floor_manager.initiate_chat(
    manager, message=
    "私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。"
)
# type exit to terminate the chat

floor_manager (to chat_manager):

私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。

--------------------------------------------------------------------------------
chef (to chat_manager):

かしこまりました。高血圧のことを考慮せず、まずは味を追求した1週間分の夕食メニューを考案します。全てが完璧に美味しい料理となるよう努めます。後ほど、医師と相談し健康面を考慮した修正を提案していただきます。

### 1週間分の夕食メニュー（味を優先）

#### 月曜日: ジュースィー・リブアイステーキ
- 肉厚なリブアイステーキ
- ガーリックバターソース
- ローストしたガーリックスキンポテト
- クリーミースピナッチ

#### 火曜日: トリュフソースの濃厚リゾット
- クリーミートリュフリゾット
- パルメザンチーズ
- グリルドマッシュルーム
- フレッシュハーブ

#### 水曜日: クラシックフレンチブーフブルギニョン
- 赤ワインで煮込んだビーフ
- マッシュポテト
- ホワイトリーハーブ
- 蒸した季節の野菜

#### 木曜日: エビとサフランのパエリア
- 大ぶりのエビとムール貝
- サフラン香るパエリアライス
- パプリカとトマト
- レモンウェッジ

#### 金曜日: 芳醇なバターチキンカレー
- クリーミーなバターチキンカレー
- 炊きたてのバスマティライス
- 香ばしいガーリックナン
- 自家製ピクルス

#### 土曜日: イタリアン・ボンゴレビアンコ
- 白ワイン蒸しのアサリ
- ガーリックスパゲッティ
- エクストラバージンオリーブオイル
- パンチェッタトッピング

#### 日曜日: テリヤキサーモンディナー
- グリルサーモンの自家製照り焼きソース
- バター醤油風味のアスパラと椎茸
- フレッシュグリーンサラダ

次は、doctorと相談し健康面の修正を行いましょう。それが完了したらfloor_managerに食材リストを提出します。

-----